In [1]:
import xarray as xr
import pandas as pd
import os
import numpy as np

import functions 

In [2]:
# columns = ['Longitude_new', 'Latitude_new', 'index', 'annual_mean_Qstl_mm', 'annual_25percent_Qstl_mm', 'annual_50percent_Qstl_mm', 'annual_75percent_Qstl_mm']

langtang = pd.read_csv('/Users/varyabazilova/Desktop/paper2/202405_modelruns/1langtangTL/default_land_cover/default_land_coverlangtang_tl_percentiles_with_coords.csv')
langtang = langtang.rename(columns = {'index': 'cellnr2'})
langtang = langtang[['cellnr2', 'Latitude_new', 'Longitude_new']]
# langtang = langtang.set_index('cellnr2')

elevation = pd.read_csv('/Users/varyabazilova/Desktop/paper2/downscaling_simple/coordinates_and_elevation_with_labels_langtang.csv')[['cellnr2','band_data']]
# elevation = elevation.set_index('cellnr2')

langtang = langtang.merge(elevation, on = 'cellnr2')
langtang = langtang.set_index('cellnr2')

location_df = langtang


# make monthly data and save to each folder 

In [3]:
path = '/Users/varyabazilova/Desktop/paper2/202405_modelruns/xrds_test/'

# df = pd.read_csv(path + 'cellnr12a/Sediment.out')


In [4]:
# df1 = functions.calculate_monthly_sediment_yield_all(df)

In [5]:
datasets = []
for folder_name in os.listdir(path):
    # skip dsstore thing
    if folder_name == '.DS_Store':
        continue
    if folder_name == '.ipynb_checkpoints':
        continue
    # skip all csvs
    if folder_name.endswith('.csv'):
        continue
    print(folder_name)
    # Extract location_id from the last 3 characters of the folder name
    location_id = folder_name[-3:]

    if location_id in location_df.index:
        location_info = location_df.loc[location_id]
        
        elevation = location_info['band_data']
        lat = location_info['Latitude_new']
        lon = location_info['Longitude_new']

        sediment_file = os.path.join(path, folder_name, 'Sediment.out')
        sediment_data = functions.calculate_monthly_sediment_yield_all(pd.read_csv(sediment_file)[['D', 'Q100']])
        sediment_data['D'] = pd.to_datetime(sediment_data.D)

          # Create xarray dataset
        ds = xr.Dataset(
            data_vars={col: (('time',), sediment_data[col]) for col in sediment_data.columns},
            coords={'time': sediment_data['D'].values}
        )

        # Convert latitude and longitude coordinates to dimensions
        ds_with_dims = ds.expand_dims({'latitude': [lat], 'longitude': [lon]}, axis=(1, 2))
        
        # datasets.append(ds_with_dims)



cellnr12b
cellnr12d
cellnr12c
cellnr13a
cellnr12a
cellnr13d
cellnr13c
cellnr13b


In [ ]:
import xarray as xr

# Assuming datasets is a list containing your datasets

# # Extract latitude and longitude values from each dataset
# latitude_values = [ds['latitude'].data[0] for ds in datasets]
# longitude_values = [ds['longitude'].data[0] for ds in datasets]

# # Create DataArrays for latitude and longitude dimensions
# latitude_dim = xr.DataArray(latitude_values, dims='location')
# longitude_dim = xr.DataArray(longitude_values, dims='location')

# Combine datasets along the time dimension
combined_ds = xr.concat(datasets, dim='time')

# Create a new dimension for latitude
# latitude_dim_combined = xr.DataArray(range(len(datasets))#, dims='location')

# Assign latitude and longitude as dimensions
# combined_ds = combined_ds.assign_coords(latitude=(('location',), latitude_dim_combined),
#                                         longitude=(('location',), longitude_dim))

combined_ds = combined_ds.assign_coords(latitude=latitude_dim, longitude=longitude_dim)

# Print the combined dataset


In [6]:
datasets

[<xarray.Dataset>
 Dimensions:    (time: 853, latitude: 1, longitude: 1)
 Coordinates:
   * latitude   (latitude) float64 28.07
   * longitude  (longitude) float64 85.53
   * time       (time) datetime64[ns] 1951-09-30 1951-10-31 ... 2022-09-30
 Data variables:
     D          (time, latitude, longitude) datetime64[ns] 1951-09-30 ... 2022...
     Q100       (time, latitude, longitude) float64 0.0 2.917e+04 ... 5.473e+04
     year       (time, latitude, longitude) int64 1951 1951 1951 ... 2022 2022
     month      (time, latitude, longitude) int64 9 10 11 12 1 2 ... 4 5 6 7 8 9,
 <xarray.Dataset>
 Dimensions:    (time: 853, latitude: 1, longitude: 1)
 Coordinates:
   * latitude   (latitude) float64 28.12
   * longitude  (longitude) float64 85.53
   * time       (time) datetime64[ns] 1951-09-30 1951-10-31 ... 2022-09-30
 Data variables:
     D          (time, latitude, longitude) datetime64[ns] 1951-09-30 ... 2022...
     Q100       (time, latitude, longitude) float64 0.0 0.0 0.0 ... 0.0

In [13]:
temp = xr.DataArray(name="temperature", data=np.random.randn(2), dims=["t"])
precip = xr.DataArray(name="precipitation", data=np.random.randn(2), dims=["t"])
# ds_grid = [[temp, precip], [temp, precip]]
ds_grid = [[temp, precip]]#, [temp, precip]]
ds_grid

[[<xarray.DataArray 'temperature' (t: 2)>
  array([-0.97264916, -0.61886148])
  Dimensions without coordinates: t,
  <xarray.DataArray 'precipitation' (t: 2)>
  array([0.21616394, 1.45705513])
  Dimensions without coordinates: t]]

In [16]:
xr.combine_nested(datasets, concat_dim=['time', 'lat', 'lon'])

ValueError: concat_dims has length 3 but the datasets passed are nested in a 1-dimensional structure

In [ ]:
# columns = ['Longitude_new', 'Latitude_new', 'index', 'annual_mean_Qstl_mm', 'annual_25percent_Qstl_mm', 'annual_50percent_Qstl_mm', 'annual_75percent_Qstl_mm']

langtang = pd.read_csv('/Users/varyabazilova/Desktop/paper2/202405_modelruns/1langtangTL/default_land_cover/default_land_coverlangtang_tl_percentiles_with_coords.csv')
langtang = langtang.rename(columns = {'index': 'cellnr2'})
langtang = langtang[['cellnr2', 'Latitude_new', 'Longitude_new']]
# langtang = langtang.set_index('cellnr2')

elevation = pd.read_csv('/Users/varyabazilova/Desktop/paper2/downscaling_simple/coordinates_and_elevation_with_labels_langtang.csv')[['cellnr2','band_data']]
# elevation = elevation.set_index('cellnr2')

langtang = langtang.merge(elevation, on = 'cellnr2')
langtang = langtang.set_index('cellnr2')

location_df = langtang


# create a datacube for mean monthly data 

In [ ]:
# read the sediment csv 
# convert to xarray 
# add coordimates and 

In [ ]:
# langtang

In [ ]:
# location_df

In [ ]:
# read one sediment file 
pathSL = '/Users/varyabazilova/Desktop/paper2/202405_modelruns/xrds_test/'
# Sed_mean = pd.read_csv(pathSL+ 'cellnr12a/Sediment.out')[['D', 'Q100']]

# Sed_mean['D'] = pd.to_datetime(Sed_mean.D)
# # extract id from the folder 

location_df = langtang

In [ ]:
# Sed_mean.dtypes

In [ ]:
# Create xarray dataset



In [ ]:
len(datasets)

In [ ]:
datasets = []
for folder_name in os.listdir(pathSL):
    # skip dsstore thing
    if folder_name == '.DS_Store':
        continue
    if folder_name == '.ipynb_checkpoints':
        continue
    # skip all csvs
    if folder_name.endswith('.csv'):
        continue
    print(folder_name)
    # Extract location_id from the last 3 characters of the folder name
    location_id = folder_name[-3:]

    if location_id in location_df.index:
        location_info = location_df.loc[location_id]
        
        elevation = location_info['band_data']
        lat = location_info['Latitude_new']
        lon = location_info['Longitude_new']

        sediment_file = os.path.join(pathSL, folder_name, 'Sediment.out')
        sediment_data = pd.read_csv(sediment_file)[['D', 'Q100']]
        sediment_data['D'] = pd.to_datetime(sediment_data.D)

          # Create xarray dataset
        ds = xr.Dataset(
            data_vars={col: (('time',), sediment_data[col]) for col in sediment_data.columns},
            coords={'time': sediment_data['D'].values}
        )

        # Convert latitude and longitude coordinates to dimensions
        ds_with_dims = ds.expand_dims({'latitude': [lat], 'longitude': [lon]}, axis=(1, 2))
        datasets.append(ds_with_dims)



In [ ]:
len(datasets)

In [ ]:
import xarray as xr

# Assuming datasets is a list containing your datasets

# Extract latitude and longitude values from each dataset
latitude_values = [ds['latitude'].data[0] for ds in datasets]
longitude_values = [ds['longitude'].data[0] for ds in datasets]

# Create DataArrays for latitude and longitude dimensions
latitude_dim = xr.DataArray(latitude_values, dims='location')
longitude_dim = xr.DataArray(longitude_values, dims='location')

# Combine datasets along the time dimension
combined_ds = xr.concat(datasets, dim='time')

# Create a new dimension for latitude
latitude_dim_combined = xr.DataArray(range(len(datasets)), dims='location')

# Assign latitude and longitude as dimensions
# combined_ds = combined_ds.assign_coords(latitude=(('location',), latitude_dim_combined),
#                                         longitude=(('location',), longitude_dim))

combined_ds = combined_ds.assign_coords(latitude=latitude_dim, longitude=longitude_dim)

# Print the combined dataset
combined_ds

In [ ]:
combined_ds.isel(time = 100).Q100.plot()